In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
import re

In [4]:
dataset = pd.read_csv('credit_score_predict.csv')
dataset.head(20)

,Loan ID,Customer ID,Current Loan Amount,Term,Credit Score,Annual Income,Years in current job,Home Ownership,Purpose,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens
0,f738779f-c726-40dc-92cf-689d73af533d,ded0b3c3-6bf4-4091-8726-47039f2c1b90,611314,Short Term,747.0,2074116.0,10+ years,Home Mortgage,Debt Consolidation,42000.83,21.8,NaN,9,0,621908,1058970,0.0,0.0
1,6dcc0947-164d-476c-a1de-3ae7283dde0a,1630e6e3-34e3-461a-8fda-09297d3140c8,266662,Short Term,734.0,1919190.0,10+ years,Home Mortgage,Debt Consolidation,36624.40,19.4,NaN,11,0,679573,904442,0.0,0.0
2,f7744d01-894b-49c3-8777-fc6431a2cff1,2c60938b-ad2b-4702-804d-eeca43949c52,153494,Short Term,709.0,871112.0,2 years,Rent,Debt Consolidation,8391.73,12.5,10.0,10,0,38532,388036,0.0,0.0
3,83721ffb-b99a-4a0f-aea5-ef472a138b41,12116614-2f3c-4d16-ad34-d92883718806,176242,Short Term,727.0,780083.0,10+ years,Rent,Debt Consolidation,16771.87,16.5,27.0,16,1,156940,531322,1.0,0.0
4,08f3789f-5714-4b10-929d-e1527ab5e5a3,39888105-fd5f-4023-860a-30a3e6f5ccb7,321992,Short Term,744.0,1761148.0,10+ years,Home Mortgage,Debt Consolidation,39478.77,26.0,44.0,14,0,359765,468072,0.0,0.0
5,a4957169-d809-44cc-847b-975400bc8d11,6878d414-6a22-4712-ae43-9b3f798e463a,202928,Short Term,741.0,760380.0,1 year,Rent,Debt Consolidation,6526.69,13.8,NaN,6,0,258647,476872,0.0,0.0
6,43467302-94fe-494b-b52f-3fd891fea71c,48113a98-a4a0-4956-b57d-f0ce344826fb,621786,Long Term,733.0,1783606.0,10+ years,Home Mortgage,Debt Consolidation,36563.98,15.3,NaN,42,0,281599,1449162,0.0,0.0
7,930c7cb3-6086-434a-9547-3ed41c181552,19941661-98e2-4800-93c9-a0e92057c813,266794,Long Term,NaN,NaN,< 1 year,Own Home,Debt Consolidation,12336.89,5.8,NaN,9,0,233206,342232,0.0,0.0
8,0b2f1b66-741e-4e37-a929-99926cdc9e9a,6a1adeda-079b-49e5-ac7c-91828f2806a0,202466,Short Term,736.0,1068617.0,5 years,Rent,Debt Consolidation,18745.21,20.5,NaN,2,0,0,0,0.0,0.0
9,d08f3a5e-93df-40e7-bdd8-cba59180bddf,4080a828-a61a-4f04-a627-397f4319500c,266288,Long Term,683.0,2031518.0,2 years,Rent,Debt Consolidation,12443.10,24.4,56.0,8,2,31445,251130,2.0,0.0


In [5]:
#Data cleaning
dataset_2 = dataset.drop(['Loan ID', 'Customer ID', 'Months since last delinquent'], axis = 'columns')
dataset_2['Years in current job'].unique()

array(['10+ years', '2 years', '1 year', '< 1 year', '5 years', '7 years',
       '3 years', '6 years', '8 years', '4 years', nan, '9 years'],
      dtype=object)

In [6]:
dataset_3 = dataset_2.copy()
dataset_3 = dataset_2.dropna()
dataset_3['Years in current job'].unique()
#removing character values
dataset_3['refined_jobYears'] = dataset_2['Years in current job'].apply( lambda x: (''.join(filter(str.isdigit, str(x)))))
#extracting integers
dataset_3['refined_jobYears'] = dataset_3['refined_jobYears'].apply( lambda x: int(x))

<ipython-input-6-3e2e83f7e569>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_3['refined_jobYears'] = dataset_2['Years in current job'].apply( lambda x: (''.join(filter(str.isdigit, str(x)))))
<ipython-input-6-3e2e83f7e569>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_3['refined_jobYears'] = dataset_3['refined_jobYears'].apply( lambda x: int(x))


In [7]:
dataset_3['refined_jobYears'].unique()

array([10,  2,  1,  5,  7,  3,  6,  8,  4,  9])

In [8]:
dataset_4 = dataset_3.replace(to_replace = ['HaveMortgage'], value = ['Home Mortgage'])
dataset_4['Home Ownership'].unique()
dataset_4.rename(columns = {'Credit Score' : 'Credit_Score','Current Credit Balance' : 'Current_Credit_Balance', 'Maximum Open Credit' : 'Maximum_Open_Credit' ,'Current Loan Amount' : 'Current_Loan_Amount', 'Annual Income' : 'Annual_income', 'Monthly Debt' : 'Monthly_Debt', 'Years of Credit History' : 'Years_of_Credit_History', 'Number of Open Accounts' : 'Number_of_Open_Accounts', 'Number of Credit Problems' : 'Number_of_Credit_Problems', 'Home Ownership' : 'Home_Ownership', 'Tax Liens' : 'Tax_Liens'}, inplace = True)
dataset_4.head(10)

,Current_Loan_Amount,Term,Credit_Score,Annual_income,Years in current job,Home_Ownership,Purpose,Monthly_Debt,Years_of_Credit_History,Number_of_Open_Accounts,Number_of_Credit_Problems,Current_Credit_Balance,Maximum_Open_Credit,Bankruptcies,Tax_Liens,refined_jobYears
0,611314,Short Term,747.0,2074116.0,10+ years,Home Mortgage,Debt Consolidation,42000.83,21.8,9,0,621908,1058970,0.0,0.0,10
1,266662,Short Term,734.0,1919190.0,10+ years,Home Mortgage,Debt Consolidation,36624.40,19.4,11,0,679573,904442,0.0,0.0,10
2,153494,Short Term,709.0,871112.0,2 years,Rent,Debt Consolidation,8391.73,12.5,10,0,38532,388036,0.0,0.0,2
3,176242,Short Term,727.0,780083.0,10+ years,Rent,Debt Consolidation,16771.87,16.5,16,1,156940,531322,1.0,0.0,10
4,321992,Short Term,744.0,1761148.0,10+ years,Home Mortgage,Debt Consolidation,39478.77,26.0,14,0,359765,468072,0.0,0.0,10
5,202928,Short Term,741.0,760380.0,1 year,Rent,Debt Consolidation,6526.69,13.8,6,0,258647,476872,0.0,0.0,1
6,621786,Long Term,733.0,1783606.0,10+ years,Home Mortgage,Debt Consolidation,36563.98,15.3,42,0,281599,1449162,0.0,0.0,10
8,202466,Short Term,736.0,1068617.0,5 years,Rent,Debt Consolidation,18745.21,20.5,2,0,0,0,0.0,0.0,5
9,266288,Long Term,683.0,2031518.0,2 years,Rent,Debt Consolidation,12443.10,24.4,8,2,31445,251130,2.0,0.0,2
11,258104,Short Term,723.0,1284514.0,7 years,Rent,Debt Consolidation,6368.99,14.6,12,0,128402,266904,0.0,0.0,7


In [9]:
dataset_5 = dataset_4.replace(to_replace = ['other', 'major_purchase', 'Home Improvements', 'vacation', 'wedding', 'Take a Trip', 'moving', 'small_business'], value = ['Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Business Loan'])
dataset_5.head(10)

,Current_Loan_Amount,Term,Credit_Score,Annual_income,Years in current job,Home_Ownership,Purpose,Monthly_Debt,Years_of_Credit_History,Number_of_Open_Accounts,Number_of_Credit_Problems,Current_Credit_Balance,Maximum_Open_Credit,Bankruptcies,Tax_Liens,refined_jobYears
0,611314,Short Term,747.0,2074116.0,10+ years,Home Mortgage,Debt Consolidation,42000.83,21.8,9,0,621908,1058970,0.0,0.0,10
1,266662,Short Term,734.0,1919190.0,10+ years,Home Mortgage,Debt Consolidation,36624.40,19.4,11,0,679573,904442,0.0,0.0,10
2,153494,Short Term,709.0,871112.0,2 years,Rent,Debt Consolidation,8391.73,12.5,10,0,38532,388036,0.0,0.0,2
3,176242,Short Term,727.0,780083.0,10+ years,Rent,Debt Consolidation,16771.87,16.5,16,1,156940,531322,1.0,0.0,10
4,321992,Short Term,744.0,1761148.0,10+ years,Home Mortgage,Debt Consolidation,39478.77,26.0,14,0,359765,468072,0.0,0.0,10
5,202928,Short Term,741.0,760380.0,1 year,Rent,Debt Consolidation,6526.69,13.8,6,0,258647,476872,0.0,0.0,1
6,621786,Long Term,733.0,1783606.0,10+ years,Home Mortgage,Debt Consolidation,36563.98,15.3,42,0,281599,1449162,0.0,0.0,10
8,202466,Short Term,736.0,1068617.0,5 years,Rent,Debt Consolidation,18745.21,20.5,2,0,0,0,0.0,0.0,5
9,266288,Long Term,683.0,2031518.0,2 years,Rent,Debt Consolidation,12443.10,24.4,8,2,31445,251130,2.0,0.0,2
11,258104,Short Term,723.0,1284514.0,7 years,Rent,Debt Consolidation,6368.99,14.6,12,0,128402,266904,0.0,0.0,7


In [10]:
#creating dummies to handle strings
dummies = pd.get_dummies(dataset_5.Home_Ownership)
dummies2 = pd.get_dummies(dataset_5.Term)
dataset_6 = pd.concat([dataset_5, dummies, dummies2], axis = 'columns')
dataset_7 = dataset_6.drop(['Years in current job', 'Home_Ownership', 'Purpose', 'Rent', 'Term', 'Short Term'], axis = 'columns')
dataset_7.rename(columns = {'Home Mortgage' : 'Home_Mortgage', 'Own Home' : 'Own_Home', 'Long Term' : 'Long_Term'})
dataset_7.dropna()

,Current_Loan_Amount,Credit_Score,Annual_income,Monthly_Debt,Years_of_Credit_History,Number_of_Open_Accounts,Number_of_Credit_Problems,Current_Credit_Balance,Maximum_Open_Credit,Bankruptcies,Tax_Liens,refined_jobYears,Home Mortgage,Own Home,Long Term
0,611314,747.0,2074116.0,42000.83,21.8,9,0,621908,1058970,0.0,0.0,10,1,0,0
1,266662,734.0,1919190.0,36624.40,19.4,11,0,679573,904442,0.0,0.0,10,1,0,0
2,153494,709.0,871112.0,8391.73,12.5,10,0,38532,388036,0.0,0.0,2,0,0,0
3,176242,727.0,780083.0,16771.87,16.5,16,1,156940,531322,1.0,0.0,10,0,0,0
4,321992,744.0,1761148.0,39478.77,26.0,14,0,359765,468072,0.0,0.0,10,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,442596,739.0,1528968.0,19494.38,16.7,11,0,419235,656876,0.0,0.0,7,1,0,0
9995,157806,731.0,1514376.0,4795.41,12.5,9,0,87058,234410,0.0,0.0,6,0,0,0
9996,132550,718.0,763192.0,12401.87,9.9,8,0,74309,329692,0.0,0.0,4,1,0,0
9998,99999999,721.0,972097.0,12232.20,16.8,8,1,184984,240658,0.0,0.0,10,1,0,0


In [11]:
#Data Training and filter data that does not affect trained output
X = dataset_7.drop(['Maximum_Open_Credit', 'Number_of_Credit_Problems', 'Long Term'], axis = 'columns')
Y = dataset_7.Maximum_Open_Credit
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.15, random_state = 15)
LR = LinearRegression()
LR.fit(X_train, Y_train)
LR.score(X_test, Y_test)
# cv = ShuffleSplit(n_splits = 5, test_size = 0.15, random_state = 15)
# cross_val_score(LinearRegression(), X, Y, cv = cv)

0.8626687450947764

In [12]:
#writing function for data prediction
def predict_credit_limit (Ownership, Current_Loan_Amount, Bankruptcies, Tax_Liens, Credit_Score, Annual_Income, Monthly_Debt, Years_of_Credit_History, Number_of_Open_Accounts, Current_Credit_Balance, refined_jobYears):
  ownership_index = np.where(X.columns == Ownership)[0]
  x = np.zeros(len(X.columns))
  x[0] = Current_Loan_Amount
  x[1] = Credit_Score
  x[2] = Annual_Income
  x[3] = Monthly_Debt
  x[4] = Years_of_Credit_History
  x[5] = Number_of_Open_Accounts
  x[6] = Current_Credit_Balance
  x[7] = Bankruptcies
  x[8] = Tax_Liens
  x[9] = refined_jobYears
  
  if ownership_index >= 0:
    x[ownership_index] = 1
  
  return LR.predict([x])[0]

In [13]:
#rechecking answers
predict_credit_limit('Own_Home', 611314, 0, 0, 747, 2074116.0, 42000.83, 21.8, 9, 621908, 10)

<ipython-input-12-536668d9ee6f>:16: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if ownership_index >= 0:
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


966359.1693648578

In [ ]:
# import pickle
# with open('Credit_limit_model.pickle', 'wb') as f:
#   pickle.dump(LR, f)

In [ ]:
# import json
# columns = {
#     'data_columns' : [col.lower() for col in X.columns]
# }
# with open("columns.json", "w") as f:
#   f.write(json.dumps(columns))

In [ ]:
#Hyperparameter Tuning
# from sklearn.linear_model import Lasso 
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.model_selection import GridSearchCV
# cv = ShuffleSplit(n_splits = 5, test_size = 0.1, random_state = 0)

# def find_best_model_using_gridsearchcv(X,Y):
#   algos = {
#     'linear regression' : {
#       'model': LinearRegression(), 
#         'params': {
#           'normalize': [True, False]
#         }
#     },
#     'lasso':{
#       'model': Lasso(),
#       'params': {
#         'alpha': [1,2],
#         'selection': ['random', 'cyclic']
#       }
#     },
#     'decision tree': { 
#       'model': DecisionTreeRegressor(),
#       'params': {
#         'criterion': ['mse', 'friedman_mse'],
#         'splitter': ['best','random']
#       }
#     }
#   }
#   scores = []
#   for algo_name, config in algos.items():
#     gs = GridSearchCV(config['model'], config['params'], cv=cv, return_train_score=True)
#     gs.fit(X,Y)
#     scores.append({
#       'model': algo_name,
#       'best score': gs.best_score_,
#       'best_params': gs.best_params_
#     })
#     return pd.DataFrame(scores, columns=['model','best_score','best_params'])

# find_best_model_using_gridsearchcv(X,Y)